In [5]:
pip install pymysql


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import requests
import pymysql
import pandas as pd
import time
import schedule


API_KEY = "82ba70773967761939a47e810ecf994489e48c2a6c39cd6dbf41573b4c9bf9c5"

categories = ["electronics", "fashion", "gaming", "home appliances", "automobile", "sports", "books", "toys"]
url = "https://serpapi.com/search"


def clean_price(value):
    if isinstance(value, str):
        value = value.replace("$", "").replace(",", "").strip()
        return float(value) if value.replace(".", "").isdigit() else 0.0
    return float(value) if isinstance(value, (int, float)) else 0.0

def connect_db():
    return pymysql.connect(
        host="localhost",
        user="saloni",
        password="Password$098",
        database="Google_shopping",
        charset="utf8mb4",
        cursorclass=pymysql.cursors.DictCursor
    )

def create_table():
    conn = connect_db()
    cursor = conn.cursor()

    table_query = """
    CREATE TABLE IF NOT EXISTS serpapi_results (
        id INT AUTO_INCREMENT PRIMARY KEY,
        title VARCHAR(255),
        Source VARCHAR(255),
        price VARCHAR(50),
        extracted_price DECIMAL(15,2),
        rating FLOAT,
        category VARCHAR(255),
        snippet VARCHAR(255),
        reviews BIGINT,
        old_price BIGINT,
        extracted_old_price DECIMAL(15,2),
        tag DECIMAL(15,2),
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );
    """
    
    cursor.execute(table_query)
    conn.commit()
    conn.close()
    print("✅ MySQL Table Verified!")

def fetch_data():
    conn = connect_db()
    cursor = conn.cursor()

    all_products = []

    for category in categories:
        print(f"🔍 Fetching data for category: {category}...")

        params = {
            "engine": "google_shopping",
            "q": category,
            "hl": "en",
            "gl": "us",
            "api_key": API_KEY
        }

        response = requests.get(url, params=params)
        data = response.json()

        products = data.get("shopping_results", [])

        for product in products:
            title = product.get("title", "N/A")
            source = product.get("source", "N/A")
            price = product.get("price", "0.0")
            extracted_price = clean_price(product.get("extracted_price", "0.0"))
            rating = clean_price(product.get("rating", "0.0"))
            snippet = product.get("snippet", "")
            reviews = int(product.get("reviews", 0))
            old_price = clean_price(product.get("old_price", "0.0"))
            extracted_old_price = clean_price(product.get("extracted_old_price", "0.0"))
            tag = clean_price(product.get("tag", "0.0"))

            all_products.append((
                title, source, price, extracted_price, rating, category, snippet,
                reviews, old_price, extracted_old_price, tag
            ))

        time.sleep(2)  s

    insert_query = """
    INSERT INTO serpapi_results 
    (title, Source, price, extracted_price, rating, category, snippet, reviews, old_price, extracted_old_price, tag)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """

    cursor.executemany(insert_query, all_products)
    conn.commit()
    conn.close()

    print("✅ Data Inserted Successfully into MySQL")


create_table()
fetch_data()

schedule.every(12).hours.do(fetch_data)

while True:
    schedule.run_pending()
    time.sleep(60)  


✅ MySQL Table Verified!
🔍 Fetching data for category: electronics...
🔍 Fetching data for category: fashion...
🔍 Fetching data for category: gaming...
🔍 Fetching data for category: home appliances...
🔍 Fetching data for category: automobile...
🔍 Fetching data for category: sports...
🔍 Fetching data for category: books...
🔍 Fetching data for category: toys...
✅ Data Inserted Successfully into MySQL
